In [ ]:
import pandas as pd
import numpy as np
from numpy.random import choice
import datetime
from datetime import datetime

In [ ]:
#People away or are already presenting
away = []
already_presenting = []

In [ ]:
nos = already_presenting + away
number_to_select = 3 - len(already_presenting) #max of three people per JC

In [ ]:
#Read in excel file
xls = pd.ExcelFile('JC_Schedule.xlsx')

In [ ]:
#Define dataframes
papers = pd.read_excel(xls, 'Schedule')
presenters = pd.read_excel(xls, 'Presenters')
groups = pd.read_excel(xls, 'Groups')

In [ ]:
#Remove unnecessary information
presenters = presenters.drop(['Count Intake Contribution', 'Count weighted by Intake Size', 'Group'], axis=1)
presenters = presenters.ffill(axis=0)
presenters = presenters.drop_duplicates()
presenters = presenters.dropna()


In [ ]:
#Remove postdocs - not compulsory 
presenters = presenters[~presenters['Intake Year'].str.contains("PostDoc", na=False)]
presenters = presenters.reset_index()

In [ ]:
#Weight
presenters['Weight'] = 1/presenters['Count Papers']
presenters = presenters.replace(np.inf, 2.0) #put higher, non inf, weight to those who have not presented 

In [ ]:
#Get weights
if len(nos) > 0:
    pattern = '|'.join(nos)
else:
    pattern = ' '
PT_ECS = '|'.join(['PT', 'ECS'])
presenters['Weight'][presenters['Name'].str.contains(pattern) == True] = 0.0 #People away or already presenting
presenters['Weight'][presenters['Notes'].str.contains(PT_ECS) == True] = 0.5*presenters['Weight'] #Weight part-time and ECS more fairly 

#ignore first years if before summer of their first year (June) - up to them to present
presenters['First summer'] = '01/06/'+(presenters['Intake Year']+1).astype(str)
format_str = '%d/%m/%Y'
b4_summer = []
for i in presenters['First summer']:
    date_obj = datetime.strptime(i, format_str)
    if datetime.today()<date_obj:
        b4_summer.append(True)
    else:
        b4_summer.append(False)
presenters['Before summer'] = b4_summer
presenters['Weight'][presenters['Before summer']==True] = 0.0 
presenters = presenters.drop(['First summer', 'Before summer'], axis=1)

In [ ]:
presenters

In [ ]:
presenters_chosen = []
#loop over the number that need to be chosen so can do probability by weight and not get duplicates
for i in range(number_to_select):
    #probability
    presenters['Probability_{}'.format(i)] = presenters['Weight']/(presenters['Weight'].sum())
    #draw at random
    draw_1 = choice(presenters['Name'], 1, p=presenters['Probability_{}'.format(i)])
    #make list of presenters
    presenters_chosen.append(draw_1[0])
    #remove person selected
    presenters = presenters.drop(presenters[presenters['Name']==draw_1[0]].index)
    print(presenters)

In [ ]:
print('Already presenting: {}'.format(already_presenting))
print('First choice presenters: {}'.format(presenters_chosen))